In [1]:
import torch
import torchvision
from torch2trt import TRTModule

In [2]:
model_trt_1 = TRTModule()
#model_trt_1.load_state_dict(torch.load('road_following_model_trt_10_08.pth'))
model_trt_1.load_state_dict(torch.load('./Models/resnet-1800_plus_trt.pth'))

<All keys matched successfully>

In [3]:
model_trt_2 = TRTModule()
model_trt_2.load_state_dict(torch.load('best_stop_model_trt_10_22.pth'))

<All keys matched successfully>

In [4]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

In [5]:
from FilesWeNeed.csi_camera import CSICamera

# liu: 感觉capture_fps = 65 过高，可能导致数据滞留，可将 fps 改为 30 进行调试，09-07号任务
camera = CSICamera(width=224, height=224, capture_fps=40)

In [20]:
car.throttle = 0

In [ ]:
camera.release_camera()

# 使用双模型法进行推理

In [ ]:
from utils import preprocess
import numpy as np
import time 
import math

STEERING_GAIN = 0.80
STEERING_BIAS = 0.00

car.throttle = 0.26

count = 0
total_time = 0
while True:
    count += 1
    begin_time = time.time()
    image = camera.read()
    image = preprocess(image).half()
    output1 = model_trt_1(image).detach().cpu().numpy().flatten()
    #output2 = np.array(model_trt_2(image).detach().cpu().numpy().flatten())
    
    if total_time > 14 and np.argmax(np.array(model_trt_2(image).detach().cpu().numpy().flatten())) == 0:
        car.throttle = 0.0
#     elif np.argmax(output2) == 0:
#         car.steering = -1.0
#         time.sleep(2)
#     elif np.argmax(output2) == 3:
#         car.throttle = 0.25
#     else: 
#         car.throttle = 0.21
    else:
        x = float(output1[0])
        car.throttle = 0.26 - abs(car.steering) * 0.04
        car.steering = x * STEERING_GAIN + STEERING_BIAS - 0.1
    end_time = time.time()
    total_time += end_time - begin_time
    print(total_time / count)

# 使用循线模型+matchTemplate法进行推理

In [19]:
from utils import preprocess
import numpy as np
import time 
import math
import cv2

stop_template = cv2.imread('stop_tem.jpg')
constraint_template = cv2.imread('constraint_tem.jpg')
unconstraint_template = cv2.imread('unconstraint_tem.jpg')


STEERING_GAIN = 0.80
STEERING_BIAS = 0.00

car.throttle = 0.22

count = 0
total_time = 0
speed_flag = 0
stop_flag = 0


while True:
    
    count += 1
    begin_time = time.time()
    image = camera.read()
    image1 = preprocess(image).half()
    output1 = model_trt_1(image1).detach().cpu().numpy().flatten()
    image = np.uint8(image)
    stop_value = cv2.minMaxLoc(cv2.matchTemplate(image, stop_template, cv2.TM_CCOEFF_NORMED))[1]
    constraint_value = cv2.minMaxLoc(cv2.matchTemplate(image, constraint_template, cv2.TM_CCOEFF_NORMED))[1]
    unconstraint_value = cv2.minMaxLoc(cv2.matchTemplate(image, unconstraint_template, cv2.TM_CCOEFF_NORMED))[1]
    
    
    
    if stop_value > 0.9:
        stop_flag = 1
        car.throttle = 0
    elif constraint_value > 0.9:
        speed_flag = 1
    elif unconstraint_value > 0.9:
        speed_flag = 0

    if stop_flag == 0 and speed_flag == 0:
        x = float(output1[0])
        car.throttle = 0.22 - abs(car.steering) * 0.04
        car.steering = x * STEERING_GAIN + STEERING_BIAS - 0.1
    elif stop_flag == 0 and speed_flag == 1:
        x = float(output1[0])
        car.throttle = 0.18
        car.steering = x * STEERING_GAIN + STEERING_BIAS - 0.1
    elif stop_flag == 1:
        car.throttle = 0
    end_time = time.time()
    total_time += end_time - begin_time
    print(total_time / count)

0.13389086723327637
0.12287342548370361
0.11803650856018066
0.1141936182975769
0.11234865188598633
0.11115280787150066
0.1102703298841204
0.10953101515769958
0.10853139559427898
0.10763761997222901
0.10699120434847745
0.1067808469136556
0.10643517054044284
0.10616217340741839
0.1059017022450765
0.10570237040519714
0.10547235432793112
0.10535685221354167
0.10511403334768195
0.10494807958602906
0.10490146137419201
0.10478451035239479
0.10479765352995499
0.10474397738774617
0.10452842712402344
0.10436577063340408
0.1043686690153899
0.10423312868390765
0.10418909993664972
0.10411636829376221
0.10395676858963505
0.10395868867635727
0.10383235324512828
0.10387267084682689
0.10399112701416016
0.10390869776407878
0.10398480698869035
0.1038679135473151
0.10382530628106533
0.10374590754508972
0.10375390983209377
0.1036831367583502
0.10370969772338867
0.10364527593959462
0.10361283090379503
0.10359318878339685
0.1035274901288621
0.10358166694641113
0.10358376405677017
0.10354688167572021
0.103530

KeyboardInterrupt: 

### 测试最佳帧率

In [ ]:
camera.release_camera()